In [2]:
import sys
import argparse
#Appendages
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/')
sys.path.append('/home/samuel.varga/python_packages/ml_workflow/')
sys.path.append('/home/samuel.varga/python_packages/VargaPy/')
# Import packages 
import pandas as pd
import numpy as np
import sklearn
from os.path import join
from sklearn.linear_model import LogisticRegression
from hyperopt import hp
from main.io import load_ml_data
from ml_workflow.calibrated_pipeline_hyperopt_cv import norm_aupdc_scorer, norm_csi_scorer
from ml_workflow.weighted_average_classifier import WeightedAverageClassifier
from ml_workflow.tuned_estimator import TunedEstimator, dates_to_groups
from VargaPy.MlUtils import All_Severe, Drop_Unwanted_Variables, Train_Ml_Parser
from sklearn.model_selection import GroupKFold
import joblib
from itertools import product

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
#Load Data
framework=['POTVIN']
timescale=['2to6']
hazard='wind'.lower() #Use tornado to get 0-2UH Baseline
SCALE=['36km','18km','9km'] #9,18,36
SigSevere=False
mode='train'
names=['hist','logistic','random','ADAM']
mod_names = ['Hist', 'Logistic', 'Random','ADAM'] 

#Load Models
model_ind=0
for FRAMEWORK, TIMESCALE, scale in product(framework, timescale, SCALE):
    save_name=f'Varga_all_wac_{hazard}_{scale}_{"SigSev" if SigSevere else "control"}_{model_ind}.joblib'
    base_path=f'/work/samuel.varga/projects/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
    data_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
    out_path=f'/work/samuel.varga/projects/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}/mlModels/{scale}'
    if hazard=='all': #Use all severe as target

        X, y, metadata = All_Severe(base_path=data_path, mode=mode, target_scale=scale[0:-2:1], 
                                    FRAMEWORK=FRAMEWORK, TIMESCALE=TIMESCALE, SigSevere=SigSevere)

    else:
        target_col=f'{hazard}_severe__{scale}'
        X,y,metadata = load_ml_data(base_path=data_path, 
                                mode=mode, 
                                target_col=target_col, FRAMEWORK=FRAMEWORK, TIMESCALE=TIMESCALE)
    X, ts_suff = Drop_Unwanted_Variables(X, original=False, training_scale=False, intrastormOnly=False, envOnly=False)
    
    # Load the ML model.
    ml_models=[]
    features=None
    for name in names:
        if name=='baseline':
            bl_model=joblib.load(join(base_path, f'blModels/{hazard}_baseline_model_control_{scale}.joblib'))
        else:
            ml_data=joblib.load(join(base_path, f'mlModels/{scale}/Varga_all_{name}_{hazard}_{scale}_{"SigSev" if SigSevere else "control"}_{model_ind}.joblib'))
            ml_model = ml_data['model']
            ml_models.append(ml_model)
            features = ml_data['X'].columns if features is None else features

    estimators = [(name, model) for name, model in zip(mod_names, ml_models)]

    X = X[features]

    #Make WAC
    train_dates=metadata['Run Date']
    groups=dates_to_groups(train_dates, n_splits=5)
    cv=list(GroupKFold(n_splits=5).split(X, y, groups))
    wac = WeightedAverageClassifier(estimators=estimators,
                         cv=cv)
    wac.fit(X, y, groups) 
    print(wac.est_names)
    print(wac.weights_)
    wac.save(join(out_path, save_name)) 

Using new variables- dropping old 90th percentile
(1274972, 174)
all


/home/samuel.varga/miniconda3/envs/Vanilla/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/home/samuel.varga/miniconda3/envs/Vanilla/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/home/samuel.varga/miniconda3/envs/Vanilla/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/home/samuel.varga/miniconda3/envs/Vanilla/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/home/samuel.varga/miniconda3/envs/Vanilla/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fit

['Hist', 'Logistic', 'Random', 'ADAM']
[0.17241095 0.14110843 0.29376882 0.3927118 ]
Using new variables- dropping old 90th percentile
(1274972, 174)
all


FileNotFoundError: [Errno 2] No such file or directory: '/work/samuel.varga/projects/2to6_hr_severe_wx/POTVIN/mlModels/18km/Varga_all_hist_wind_18km_control_0.joblib'

In [ ]:
Varga_all_logistic_wind_36km_control_0.joblib